15/03/2025

In [2]:
import pandas as pd

In [3]:
df_clienti = pd.read_csv(r"C:\Users\enduser\Desktop\PythonPostLaurea\ZOPPIS\ZOPPIS\data\clienti.csv",sep = ";")


### Full text creation

In [18]:
import ollama
import json
import os

In [ ]:
def create_full_text(row, prefix="search_document"):
    return f"{prefix}: " + ", ".join(
        f"{col}: {val}"
        for col, val in row.items() 
        if pd.notna(val)
    )

df_clienti["full_text"] = df_clienti.apply(create_full_text, axis=1)

In [16]:
df_clienti_full_text = df_clienti["full_text"]

### Embedding generation

NB: sulla repo di nomic-embed-text dicono che bisogna mettere una chiave per dire che tipo di task si vuole fare.

In [ ]:
def get_embedding(text):
    try:
        response = ollama.embeddings(model="nomic-embed-text", prompt=text)
        return response['embedding']
    except Exception as e:
        print(f"Errore durante l'embedding: {e}")
        return None

path = r"C:\Users\enduser\Desktop\PythonPostLaurea\ZOPPIS\ZOPPIS\notebooks\embeddings.parquet"

if not os.path.exists(path):
    df_clienti_emb = df_clienti_full_text.apply(get_embedding)
    df_clienti_emb = pd.DataFrame(df_clienti_emb)
    df_clienti_emb.to_parquet("embeddings_v2.parquet", engine="pyarrow")
    print("Embeddings salvati")
else:
    print(f"Il path '{path}' esiste già, uscita dal ciclo.")


Il path 'C:\Users\enduser\Desktop\PythonPostLaurea\ZOPPIS\ZOPPIS\notebooks\embeddings.parquet' esiste già, uscita dal ciclo.


ci ha messo 13 min x 6600 clienti.

la dimensione è di 768

### OCR:

con gemini flash

In [ ]:
import google.generativeai as genai
import os
import base64
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.environ.get("GOOGLE_API_KEY")
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel("gemini-2.0-flash")
doc_path = r"C:\Users\enduser\Desktop\PythonPostLaurea\ZOPPIS\ZOPPIS\data\inputs\bomaki.pdf"
with open(doc_path, "rb") as doc_file:
    doc_data = base64.standard_b64encode(doc_file.read()).decode("utf-8")
prompt = """
Trascrivi fedelmente il contenuto di questo documento PDF. Non aggiungere spiegazioni, riassunti o commenti.
Devi solo riportare il testo esattamente come appare nel documento.
"""
response = model.generate_content([
    {'mime_type': 'application/pdf', 'data': doc_data},
    prompt
])
#print(response.text)
ordine = response.text


BOMAKI
URAMAKERIA NIPPO BRASILIANA

Ordine di
acquisto
Cod. Fornitore
F100143
Numero
230032104
P.IVA/Cod. fisc.
IT01576310039
Data
13/12/2024

Vs. Riferimento
Valuta
EUR
Agente
Pag.
1 of 1
Condizioni e modalità di pagamento
Bonifico SEPA 30 GG DF FM

10I Srl
Sede legale Corso Sempione 20154 Milano (MI) - Italia
P.IVA/C Fiscale IT04995690965
R.Ε.Α. 561011
Tel. 0289071703
Email: amministrazione@bomaki.it
Web: www.bomaki.it

Spett.le
ZOPPIS SRL
VIA PER ALZO 21
28017 SAN MAURIZIO D'OPAGLIO (NO) -
ITALY

Indirizzo di spedizione
Bomaki di:
Via Bernardino Corio 8
20135 MILANO
ITALYVia Bernardino Corio 8
20135 MILANO

Cod. articolo Descrizione articolo Qta U.M. Prezzo netto Cod.IVA Imponibile
19675 ACQUA BRILLANTE TONICA VAP 1.00 CT 11.95 22.00 14.58
15061 CACHACA LEBLON 70 MAISON LEBLON 6.00 CT 603.00 22.00 735.66
8729 GIN HENDRICK'S CL100 2.00 PZ 65.90 22.00 80.40
15666 HIERBAS DE IBIZA Litro AROMATICAS DE IBIZA! 12.00 PZ 203.40 22.00 248.15
24626 SAMBUCA RAMAZZOTI CL70 3.00 PZ 20.85 22.00 2

### RAG

In [ ]:
df_clienti_emb = pd.read_parquet(r"C:\Users\enduser\Desktop\PythonPostLaurea\ZOPPIS\ZOPPIS\notebooks\embeddings_v2.parquet")
query = "search_query: Trovami l'IDCliente (Codice Identificatore Cliente) del seguente ordine: \n" + ordine
query_emb = get_embedding(query)

In [54]:
import numpy as np
from scipy.spatial.distance import cdist
k = 15
embeddings_series = df_clienti_emb["full_text"]
query_embedding = query_emb  
embedding_matrix = np.array(embeddings_series.tolist(), dtype=np.float32)
print("Shape della matrice degli embedding:", embedding_matrix.shape)
print("Shape dell'embedding della query:", len(query_embedding))
similarities = 1 - cdist([query_embedding], embedding_matrix, metric="cosine")[0]
top_k_indices = np.argsort(similarities)[::-1][:k]
print("Top 5 Indici più simili:", top_k_indices)
print("Similarità corrispondenti:", similarities[top_k_indices])

rag_results = "\n".join(df_clienti_full_text.loc[top_k_indices].astype(str).tolist())

Shape della matrice degli embedding: (6620, 768)
Shape dell'embedding della query: 768
Top 5 Indici più simili: [4128 5703 4103 3830 4104 3829 5490 5569 4555 3828 4923 4780 5957 3541
 6560]
Similarità corrispondenti: [0.7846898  0.78333956 0.77841131 0.76946964 0.76844218 0.76806016
 0.76721181 0.76580816 0.76169685 0.76137881 0.75861737 0.755783
 0.75376836 0.75115813 0.74996912]


In [55]:
rag_results = "\n".join(df_clienti_full_text.loc[top_k_indices].astype(str).tolist())
prompt_rag = f"""search_query: Trovami l'IDCliente (Codice Identificatore Cliente) del seguente ordine:

🚀 **IMPORTANTE: INIZIO ORDINE** 🚀

\nBOMAKI\nURAMAKERIA NIPPO BRASILIANA\nOrdine di\nacquisto\nCod. Fornitore\nF100143\nVs. Riferimento\nAgente\nCondizioni e modalità di pagamento\nBonifico SEPA 30 GG DF FM\nNumero\n230032104\nP.IVA/Cod. fisc.\nIT01576310039\nData\n13/12/2024\nValuta\nEUR\nPag.\n1 of 1\n101 Srl\nSede legale Corso Sempione 20154 Milano (MI) - Italia\nP.IVA/C Fiscale IT04995690965\nR.Ε.Α. 561011\nTel. 0289071703\nEmail: amministrazione@bomaki.it\nWeb: www.bomaki.it\nSpett.le\nZOPPIS SRL\nVIA PER ALZO 21\n28017 SAN MAURIZIO D'OPAGLIO (NO) -\nITALY\nIndirizzo di spedizione\nBomaki di:\nVia Bernardino Corio 8\n20135 MILANO\nITALYVia Bernardino Corio 8\n20135 MILANO\n\nCod. articola Descrizione articolo Qta U.M. Prezzo netto Cod. IVA Imponibile\n19675 ACQUA BRILLANTE TONICA VAP 1.00 CT 11.95 22.00 14.58\n15061 CACHACA LEBLON 70 MAISON LEBLON 6.00 CT 603.00 22.00 735.66\n8729 GIN HENDRICK'S CL100 2.00 PZ 65.90 22.00 80.40\n15666 HIERBAS DE IBIZA Litro AROMATICAS DE IBIZA! 12.00 PZ 203.40 22.00 248.15\n24626 SAMBUCA RAMAZZOTI CL70 3.00 PZ 20.85 22.00 25.44\n24945 SHARIS FELLUGA VENEZIE 30.00 PZ 289.50 22.00 353.19\n13035 TEQUILA PATRON Silver CL70 2.00 PZ 67.50 22.00 82.35\n20577 VODKA 42 BELOW LITRO 30.00 PZ 436.50 22.00 532.53\n

 **IMPORTANTE: FINE ORDINE**

 "sapendo che tramite RAG ho ottenuto i full text (stringa contenente tutti i dati strutturati tabellari) dei clienti più simili all'ordine in oggetto. Riporto di seguito i dati full text dei clienti più simili all'ordine. Dai importanza sopratutto alla destinazione dell'ordine e alla ragione sociale dell'ordine (l'Indirizzo Dest la RagSoc nella tabella) per scegliere la riga trai top-k che ti propongo! \n"
  
 🚀 **IMPORTANTE: INIZIO ELENCO DATI ESTRATTI TRAMITE RAG** 🚀
 
 {rag_results}

  **IMPORTANTE: FINE ELENCO DATI ESTRATTI TRAMITE RAG**

"\nRestituiscimi soltanto l'IDCliente, la RagSoc e l'Indirizzo Dest in formato JSON, non voglio alcun tipo di dettaglio aggiuntivo"" 
"""

In [61]:
def count_tokens_simple(text):
    return len(text.split())

print(count_tokens_simple(prompt_rag))


1761


In [60]:
print(prompt_rag)

search_query: Trovami l'IDCliente (Codice Identificatore Cliente) del seguente ordine:

🚀 **IMPORTANTE: INIZIO ORDINE** 🚀


BOMAKI
URAMAKERIA NIPPO BRASILIANA
Ordine di
acquisto
Cod. Fornitore
F100143
Vs. Riferimento
Agente
Condizioni e modalità di pagamento
Bonifico SEPA 30 GG DF FM
Numero
230032104
P.IVA/Cod. fisc.
IT01576310039
Data
13/12/2024
Valuta
EUR
Pag.
1 of 1
101 Srl
Sede legale Corso Sempione 20154 Milano (MI) - Italia
P.IVA/C Fiscale IT04995690965
R.Ε.Α. 561011
Tel. 0289071703
Email: amministrazione@bomaki.it
Web: www.bomaki.it
Spett.le
ZOPPIS SRL
VIA PER ALZO 21
28017 SAN MAURIZIO D'OPAGLIO (NO) -
ITALY
Indirizzo di spedizione
Bomaki di:
Via Bernardino Corio 8
20135 MILANO
ITALYVia Bernardino Corio 8
20135 MILANO

Cod. articola Descrizione articolo Qta U.M. Prezzo netto Cod. IVA Imponibile
19675 ACQUA BRILLANTE TONICA VAP 1.00 CT 11.95 22.00 14.58
15061 CACHACA LEBLON 70 MAISON LEBLON 6.00 CT 603.00 22.00 735.66
8729 GIN HENDRICK'S CL100 2.00 PZ 65.90 22.00 80.40
15666 HIE

In [57]:
generation_config = {
  "temperature": 0.1,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "application/json", # posso richiedere esplicitamente il formato strutturato della risposta
}

model = genai.GenerativeModel(
    model_name="gemini-2.0-flash-exp",
    generation_config=generation_config,
    #system_instruction="Sei un venditore, quindi i campi vuoti non li sostituisci con Null ma con valori inventati!" # solo come esperimento..
)

output_rag = model.generate_content(prompt_rag)
print(output_rag.text)

[
  {
    "IDCliente": "9550",
    "RagSoc": "IOI SRL - BOMAKI PORTA ROMANA",
    "Indirizzo_Dest": "VIA BERNARDINO CORIO, 8"
  }
]
